In [12]:
import pykep as pk
import numpy as np
pk.util.load_spice_kernel("/home/m/mmk40/Desktop/Planet Position/de431.bsp")
from pykep.orbit_plots import plot_planet, plot_lambert
from pykep import AU, DAY2SEC, SEC2DAY, DAY2YEAR
import pygmo as pg
from datetime import datetime, timedelta
import math

In [13]:
# Load planets
jupiter = pk.planet.spice('JUPITER BARYCENTER', 'SUN', 'ECLIPJ2000', 'NONE' )

# Set launch window
start_date = pk.epoch_from_string("2045-Jun-07 00:00:00")
end_date = pk.epoch_from_string("2050-Jan-01 00:00:00")

# Parameters
launch_step = 365    # days between launch trials
tof_range = (12000, 16000)  # in days, time of flight range

In [14]:
# Converting ecliptic coordinates to cartesian co ordinates & Defining the position of the Heliosphere Nose

def ecliptic_to_cartesian(lambda_deg, beta_deg, distance_km):
    lam_rad = np.deg2rad(lambda_deg)
    beta_rad = np.deg2rad(beta_deg)

    x = distance_km * np.cos(beta_rad) * np.cos(lam_rad) #cos(lat)cos(long)
    y = distance_km * np.cos(beta_rad) * np.sin(lam_rad)
    z = distance_km * np.sin(beta_rad)
    return np.array([x, y, z])


lam = 240 # Latitude of the Nose from Freeman's
beta = 7.5 # Longitude of the Nose from Freeman's
heliopause_distance = 120*pk.AU 

rH = ecliptic_to_cartesian(lam, beta, heliopause_distance) # position of the nose of the helioshpere
#print("Cartesian position (heliocentric, J2000):", rH)

In [15]:
# Results list
results = []

# Scan through departure dates
t_dep = start_date
while t_dep.mjd2000 < end_date.mjd2000:
    rJ, vJ = jupiter.eph(t_dep)

    for tof_days in range(tof_range[0], tof_range[1], 30):  # try multiple TOFs
        try:
            t_arr = pk.epoch(t_dep.mjd2000 + tof_days)
            rH = ecliptic_to_cartesian(lam, beta, heliopause_distance)

            dt_sec = tof_days * pk.DAY2SEC
            l = pk.lambert_problem(r1=rJ, r2=rH, tof=dt_sec, mu=pk.MU_SUN, max_revs=2)

            v1 = np.array(l.get_v1()[0])
            v2 = np.array(l.get_v2()[0])
            v_dep = (v1 - np.array(vJ))/1000
            delta_v = (np.linalg.norm(v1 - np.array(vJ)))/1000 
            v_arrival   = np.linalg.norm(v2)/1000  # velocity of spacecraft at heliopause arrival

            # Get Keplerian elements of transfer orbit
            kep = pk.ic2par(rJ, v1, pk.MU_SUN)  # r, v at arrival → orbital elements

            results.append({
                'departure': str(t_dep),
                'arrival': str(t_arr),
                'tof_days': tof_days,
                'sma': kep[0],   # semi-major axis
                'ecc': kep[1],   # eccentricity
                'inc': np.degrees(kep[2]),  # inclination (rad → deg)
                'raan': np.degrees(kep[3]),
                'argp': np.degrees(kep[4]),
                'ta': np.degrees(kep[5]),
                })

        except RuntimeError as e:
            # Lambert solver may fail for some configurations; skip them
            continue

    t_dep = pk.epoch(t_dep.mjd2000 + launch_step)

In [17]:
# Print trajectories
for res in results[:]:
    print(f"\033[1mLaunch,\033[0m {res['departure']}, \033[1mArrival,\033[0m {res['arrival']}, \033[1mTime to Reach Heliopause,\033[0m {res['tof_days']/365 : .2f} years")
    #print(f"Elements for JUPITER -> HELIOPAUSE → SMA: {res['sma']:.3f} km, ECC: {res['ecc']:.4f}, INC: {res['inc']:.2f}°\n")
    #print(f"Elements JUPITER -> HELIOPAUSE → RAAN: {res['raan']:.3f}° , AP: {res['argp']:.4f}°, TA: {res['ta']:.2f}°\n\n\n")

Launch, 2045-Jun-07 00:00:00, Arrival, 2078-Apr-15 00:00:00, Time to Reach Heliopause,  32.88 years
Launch, 2045-Jun-07 00:00:00, Arrival, 2078-May-15 00:00:00, Time to Reach Heliopause,  32.96 years
Launch, 2045-Jun-07 00:00:00, Arrival, 2078-Jun-14 00:00:00, Time to Reach Heliopause,  33.04 years
Launch, 2045-Jun-07 00:00:00, Arrival, 2078-Jul-14 00:00:00, Time to Reach Heliopause,  33.12 years
Launch, 2045-Jun-07 00:00:00, Arrival, 2078-Aug-13 00:00:00, Time to Reach Heliopause,  33.21 years
Launch, 2045-Jun-07 00:00:00, Arrival, 2078-Sep-12 00:00:00, Time to Reach Heliopause,  33.29 years
Launch, 2045-Jun-07 00:00:00, Arrival, 2078-Oct-12 00:00:00, Time to Reach Heliopause,  33.37 years
Launch, 2045-Jun-07 00:00:00, Arrival, 2078-Nov-11 00:00:00, Time to Reach Heliopause,  33.45 years
Launch, 2045-Jun-07 00:00:00, Arrival, 2078-Dec-11 00:00:00, Time to Reach Heliopause,  33.53 years
Launch, 2045-Jun-07 00:00:00, Arrival, 2079-Jan-10 00:00:00, Time to Reach Heliopause,  33.62 years
